In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
!tar xf spark-3.5.3-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,566 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,631 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,517

In [3]:
import os
import sys
import findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.3-bin-hadoop3"

# tornar o pyspark "importável"
findspark.init()
findspark.find()

'/content/spark-3.5.3-bin-hadoop3'

In [35]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Exemplo DataFrame") \
    .getOrCreate()


dados = [
    ("joe", "mango", 2),
    ("kevin", "kiwi", 2),
    ("joe", "banana", 2),
    ("joe", "mango", 3),
    ("kevin", "taco", 2),
    ("kevin", "kiwi", 2),
    ("joe", "apple", 4),
    ("kevin", "taco", 3),
    ("joe", "mango", 1),
    ("kevin", "kiwi", 3),
    ("joe", "banana", 1),
    ("kevin", "peach", 2)
]


colunas = "nome", "item", "peso"


df = spark.createDataFrame(dados, schema=colunas)

print("--- Dataframe Criado ---")
df.show()
print('\t')
print("--- Schema do Dataframe ---")
df.printSchema()

--- Dataframe Criado ---
+-----+------+----+
| nome|  item|peso|
+-----+------+----+
|  joe| mango|   2|
|kevin|  kiwi|   2|
|  joe|banana|   2|
|  joe| mango|   3|
|kevin|  taco|   2|
|kevin|  kiwi|   2|
|  joe| apple|   4|
|kevin|  taco|   3|
|  joe| mango|   1|
|kevin|  kiwi|   3|
|  joe|banana|   1|
|kevin| peach|   2|
+-----+------+----+

	
--- Schema do Dataframe ---
root
 |-- nome: string (nullable = true)
 |-- item: string (nullable = true)
 |-- peso: long (nullable = true)



In [32]:
df.createOrReplaceTempView("registro")


In [33]:
spark.sql("select * from registro").show()

+-----+------+----+
| nome|  item|peso|
+-----+------+----+
|  joe| mango|   2|
|kevin|  kiwi|   2|
|  joe|banana|   2|
|  joe| mango|   3|
|kevin|  taco|   2|
|kevin|  kiwi|   2|
|  joe| apple|   4|
|kevin|  taco|   3|
|  joe| mango|   1|
|kevin|  kiwi|   3|
|  joe|banana|   1|
|kevin| peach|   2|
+-----+------+----+



In [34]:
spark.sql("""
          SELECT nome, item, SUM(peso) AS peso_total
            FROM registro
          GROUP BY nome, item
          ORDER BY nome""").show()

+-----+------+----------+
| nome|  item|peso_total|
+-----+------+----------+
|  joe| mango|         6|
|  joe|banana|         3|
|  joe| apple|         4|
|kevin|  kiwi|         7|
|kevin|  taco|         5|
|kevin| peach|         2|
+-----+------+----------+

